In [ ]:
import csv
from lxml import etree
'''from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')'''

header = ['Title', 'Text']
f= open("file.csv", "w+")
writer = csv.writer(f)
writer.writerow(header)
lista = []

totalCount = 0
for event, elem in etree.iterparse("/home/fabbro/Documents/Cartella/fileWiki.xml"):
    text = ""
    title=""
    if('title' in str(elem.tag)):
        #title_list.append(elem.text)
        #titoliTxt.write(elem.text+"\n")
        if(len(str(elem.text)) > 0):
            title = elem.text
            lista.append(title)
    if('text' in str(elem.tag)):
        #text_list.append(elem.text)
        if(len(str(elem.text)) > 0):
            #paroleTxt.write(elem.text+"\n")
            text = elem.text
            '''stop_words = set(stopwords.words('english'))
            word_tokens = word_tokenize(elem.text)
            
            filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
            for w in word_tokens:
                if w not in stop_words:
                    filtered_sentence.append(w)
            print(str(filtered_sentence))'''
            #lista.append(str(filtered_sentence))
            lista.append(text)
    if len(lista) == 2:
        writer.writerow([lista[0], lista[1]])
        lista=[]
    elem.clear()
    totalCount += 1
    if totalCount > 1 and (totalCount % 100000) == 0:
            print("{:,}".format(totalCount)+" rows written of 356,900,000")
    '''if totalCount > 35000:
        break'''
f.close()
print("-------------------------------------------DONE------------------------------------------------------")


---

Various imports

In [16]:
import csv
from lxml import etree
import nltk
from nltk.corpus import stopwords
import mwparserfromhell
import re
import os.path
from progress.bar import Bar
from tqdm import tqdm

Static things

In [4]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
header = ['Title', 'Text']
lista = []

[nltk_data] Downloading package stopwords to /home/fabbro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Dinamic things

In [ ]:
f= open("file.csv", "w+")
writer = csv.writer(f)
writer.writerow(header)
totalCount = 0
file_xml= ''
if os.path.isfile("/home/fabbro/Documents/Cartella/fileWiki.xml"):
    file_xml = "/home/fabbro/Documents/Cartella/fileWiki.xml"
elif os.path.isfile("/home/fabbro/Documents/Big-Data/file.xml"):
    file_xml = "/home/fabbro/Documents/Big-Data/file.xml"

for event, elem in etree.iterparse(file_xml):
    text = ""
    title=""
    if('title' in str(elem.tag)):
        #title_list.append(elem.text)
        #titoliTxt.write(elem.text+"\n")
        if(elem.text is not None):
            title = elem.text.lower()
            title = " ".join([word for word in title.split() if word not in stop_words])
            lista.append(title)
            #print(title)

    if('text' in str(elem.tag)):
        #text_list.append(elem.text)
        if(elem.text is not None):
            text = elem.text.lower()
            text = " ".join([word for word in text.split() if word not in stop_words])
            lista.append(text)
            #print(text)
            
    if len(lista) == 2:
        writer.writerow([lista[0], lista[1]])
        lista=[]
    elem.clear()
    totalCount += 1
    if totalCount > 1 and (totalCount % 100000) == 0:
            print("{:,}".format(totalCount)+" rows written of 356,900,000")
    '''if totalCount > 35000:
        break'''
f.close()
print("-------------------------------------------DONE------------------------------------------------------")


---

Removing the tags with the class 'mwparserfromhell'

In [20]:
pip install mwparserfromhell

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [63]:
import mwparserfromhell

wikitext = '''The quick brown [[fox]] jumps over the lazy [[dog]].
[[Category:Animals]]
== See also ==
* [[Fox]]
* [[Dog]]
* [[Lion]]'''
'''
parsed_wikitext = parser(wikitext)

for element in parsed_wikitext.filter_wikilinks():
    print(element)'''

text = "I has a template! {{foo|bar|baz|eggs=spam}} See it? [[fox]]"
wikicode = mwparserfromhell.parse(text)
print(wikicode)
templates = wikicode.filter_templates()
text = " ".join([testo for testo in text.split() if testo not in templates])
print(text)
'''text_list = text.split(' ')
for x in range(len(text_list)):
    if text_list[x] not in templates:
        print(text_list[x])'''

I has a template! {{foo|bar|baz|eggs=spam}} See it? [[fox]]
I has a template! See it? [[fox]]


"text_list = text.split(' ')\nfor x in range(len(text_list)):\n    if text_list[x] not in templates:\n        print(text_list[x])"

----
Removing the sections from the wikipedia dump

In [36]:
import re

testo = ''' New York is a state in the Northeastern United States and is the 27th-most extensive, fourth-most populous, and seventh-most densely populated U.S. state. New York is bordered by New Jersey and Pennsylvania to the south and Connecticut, Massachusetts, and Vermont to the east. The state has a maritime border in the Atlantic Ocean with Rhode Island, east of Long Island, as well as an international border with the Canadian provinces of Quebec to the north and Ontario to the west and north. The state of New York, with an estimated 19.8 million residents in 2015, is often referred to as New York State to distinguish it from New York City, the state's most populous city and its economic hub.
With an estimated population of 8.55 million in 2015, New York City is the most populous city in the United States and the premier gateway for legal immigration to the United States. The New York City Metropolitan Area is one of the most populous urban agglomerations in the world. New York City is a global city, exerting a significant impact upon commerce, finance, media, art, fashion, research, technology, education, and entertainment, its fast pace defining the term New York minute. The home of the United Nations Headquarters, New York City is an important center for international diplomacy and has been described as the cultural and financial capital of the world, as well as the world's most economically powerful city. New York City makes up over 40% of the population of New York State. Two-thirds of the state's population lives in the New York City Metropolitan Area, and nearly 40% live on Long Island. Both the state and New York City were named for the 17th century Duke of York, future King James II of England. The next four most populous cities in the state are Buffalo, Rochester, Yonkers, and Syracuse, while the state capital is Albany.
The earliest Europeans in New York were French colonists and Jesuit missionaries who arrived southward from settlements at Montreal for trade and proselytizing. New York had been inhabited by tribes of Algonquian and Iroquoian-speaking Native Americans for several hundred years by the time Dutch settlers moved into the region in the early 17th century. In 1609, the region was first claimed by Henry Hudson for the Dutch, who built Fort Nassau in 1614 at the confluence of the Hudson and Mohawk rivers, where the present-day capital of Albany later developed. The Dutch soon also settled New Amsterdam and parts of the Hudson Valley, establishing the colony of New Netherland, a multicultural community from its earliest days and a center of trade and immigration. The British annexed the colony from the Dutch in 1664. The borders of the British colony, the Province of New York, were similar to those of the present-day state.
Many landmarks in New York are well known to both international and domestic visitors, with New York State hosting four of the world's ten most-visited tourist attractions in 2013: Times Square, Central Park, Niagara Falls (shared with Ontario), and Grand Central Terminal. New York is home to the Statue of Liberty, a symbol of the United States and its ideals of freedom, democracy, and opportunity. In the 21st century, New York has emerged as a global node of creativity and entrepreneurship, social tolerance, and environmental sustainability. New York's higher education network comprises approximately 200 colleges and universities, including Columbia University, Cornell University, New York University, and Rockefeller University, which have been ranked among the top 35 in the world.


== History ==


=== 16th century ===
In 1524, Giovanni da Verrazzano, an Italian explorer in the service of the French crown, explored the Atlantic coast of North America between the Carolinas and Newfoundland, including New York Harbor and Narragansett Bay. On April 17, 1524 Verrazanno entered New York Bay, by way of the Strait now called the Narrows into the northern bay which he named Santa Margherita, in honour of the King of France's sister. Verrazzano described it as "a vast coastline with a deep delta in which every kind of ship could pass" and he adds: "that it extends inland for a league and opens up to form a beautiful lake. This vast sheet of water swarmed with native boats". He landed on the tip of Manhattan and perhaps on the furthest point of Long Island. Verrazanno's stay in this place was interrupted by a storm which pushed him north towards Martha's Vineyard.
In 1540 French traders from New France built a chateau on Castle Island, within present-day Albany; due to flooding, it was abandoned the next year. In 1614, the Dutch under the command of Hendrick Corstiaensen, rebuilt the French chateau, which they called Fort Nassau. Fort Nassau was the first Dutch settlement in North America, and was located along the Hudson River, also within present-day Albany. The small fort served as a trading post and warehouse. Located on the Hudson River flood plain, the rudimentary "fort" was washed away by flooding in 1617, and abandoned for good after Fort Orange (New Netherland) was built nearby in 1623.


=== 17th century ===

Henry Hudson's 1609 voyage marked the beginning of European involvement with the area. Sailing for the Dutch East India Company and looking for a passage to Asia, he entered the Upper New York Bay on September 11 of that year. Word of his findings encouraged Dutch merchants to explore the coast in search for profitable fur trading with local Native American tribes.
During the 17th century, Dutch trading posts established for the trade of pelts from the Lenape, Iroquois, and other tribes were founded in the colony of New Netherland. The first of these trading posts were Fort Nassau (1614, near present-day Albany); Fort Orange (1624, on the Hudson River just south of the current city of Albany and created to replace Fort Nassau), developing into settlement Beverwijck (1647), and into what became Albany; Fort Amsterdam (1625, to develop into the town New Amsterdam which is present-day New York City); and Esopus, (1653, now Kingston). The success of the patroonship of Rensselaerswyck (1630), which surrounded Albany and lasted until the mid-19th century, was also a key factor in the early success of the colony. The English captured the colony during the Second Anglo-Dutch War and governed it as the Province of New York. The city of New York was recaptured by the Dutch in 1673 during the Third Anglo-Dutch War (1672–1674) and renamed New Orange. It was returned to the English under the terms of the Treaty of Westminster a year later.


== References ==


== Further reading ==

French, John Homer (1860). Historical and statistical gazetteer of New York State. Syracuse, New York: R. Pearsall Smith. OCLC 224691273. (Full text via Google Books.)
New York State Historical Association (1940). New York: A Guide to the Empire State. New York City: Oxford University Press. ISBN 978-1-60354-031-5. OCLC 504264143. (Full text via Google Books.)


== External links ==
New York at DMOZ
 Geographic data related to New York at OpenStreetMap'''


section_title_re = re.compile("^=+\s+.*\s+=+$")
content = []
skip = False
for l in testo.splitlines():
    line = l.strip()
    if "= references =" in line.lower():
        skip = True  # replace with break if this is the last section
        break
    if "= further reading =" in line.lower():
        skip = True  # replace with break if this is the last section
        continue
    if section_title_re.match(line):
        skip = False
        continue
    if skip:
        continue
    content.append(line)

i=0
for x in content:
    if x != "":
        print(x.lower())
        print('-----------------------------')

new york is a state in the northeastern united states and is the 27th-most extensive, fourth-most populous, and seventh-most densely populated u.s. state. new york is bordered by new jersey and pennsylvania to the south and connecticut, massachusetts, and vermont to the east. the state has a maritime border in the atlantic ocean with rhode island, east of long island, as well as an international border with the canadian provinces of quebec to the north and ontario to the west and north. the state of new york, with an estimated 19.8 million residents in 2015, is often referred to as new york state to distinguish it from new york city, the state's most populous city and its economic hub.
-----------------------------
with an estimated population of 8.55 million in 2015, new york city is the most populous city in the united states and the premier gateway for legal immigration to the united states. the new york city metropolitan area is one of the most populous urban agglomerations in the 

----
Idee per andare avanti:
1. Eliminare prima le doppie graffe (di apertura e chiusura) eliminando il loro contenuto

2. Capire come gestire il comportamento delle stringhe tipo `[[political movement|movement]]` dove l'importante è contenuto dopo la pipe.

3. Eliminare qualsiasi segno di punteggiature (creando una lista di tutti i simboli di punteggiatura) andandoli a rimpiazzare con un singolo spazio in modo che ogni parola sia distanziata effettivamente dall'altra

4. Effettuare un replace di tutti gli spazi (ce ne serve solo uno tra una parola e l'altra, non di più), basta usare la classe `re`

5. Ora che abbiamo tutto l'occorrente, salvare su un file csv ed effettuare l'analisi con _pyspark_

----
_Domande al professore:_

__Risposte da scrivere nella documentazione finale__

1. I numeri sono visti come parole? Sì

2. Teniamo conto dei titoli dei sotto-capitoli? No

3. Teniamo conto delle ''references''? No

4. Posso usare PonyORM?

----

In [5]:
def removeNestedParentheses(s):
    ret = ''
    skip = 0
    for i in s:
        if i == '{':
            skip += 1
        elif i == '}'and skip > 0:
            skip -= 1
        elif skip == 0:
            ret += i
    return ret

In [20]:
f= open("file.csv", "w+")
writer = csv.writer(f)
writer.writerow(header)
totalCount = 0
section_title_re = re.compile("^=+\s+.*\s+=+$")
skip = False
lista = []
file_xml= ''

if os.path.isfile("/home/fabbro/Documents/Cartella/fileWiki.xml"):
    file_xml = "/home/fabbro/Documents/Cartella/fileWiki.xml"
elif os.path.isfile("/home/fabbro/Documents/Big-Data/file.xml"):
    file_xml = "/home/fabbro/Documents/Big-Data/file.xml"

with tqdm(total=100) as pbar:
    for event, elem in etree.iterparse(file_xml):
        text = ""
        title=""
        if('title' in str(elem.tag)):
            if(elem.text is not None):
                title = elem.text.lower()
                lista.append(title)

        if('text' in str(elem.tag)):
            if(elem.text is not None):
                text = elem.text.lower()
                '''
                wikicode = mwparserfromhell.parse(text)
                templates = wikicode.filter_templates()
                '''
                '''if "= references =" in text_list[x].lower():
                            skip = True  # replace with break if this is the last section
                            break
                        if "= further reading =" in text_list[x].lower():
                            skip = True  # replace with break if this is the last section
                            continue
                        if section_title_re.match(line):
                            skip = False
                            continue
                        if skip:
                            continue'''
                text = " ".join([word for word in text.split() if word not in stop_words])
                text = removeNestedParentheses(text)
                #print(str(text))           
        if len(lista) == 2:
            writer.writerow([lista[0], lista[1]])
            lista=[]
        elem.clear()
        totalCount += 1
        if totalCount > 1 and (totalCount % 356900000000) == 0:
            print("{:,}".format(totalCount)+" rows written of 100")
            pbar.update(1)

f.close()
print("-------------------------------------------DONE------------------------------------------------------")

  0%|          | 0/100 [04:26<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/home/fabbro/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3621/4277889974.py", line 43, in <module>
    text = removeNestedParentheses(text)
  File "/tmp/ipykernel_3621/2135454401.py", line -1, in removeNestedParentheses
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/fabbro/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/fabbro/.local/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/fabbro/.local/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1012, in structured_traceback
    return VerboseTB.structured_tr

In [1]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 10.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 23.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845509 sha256=4f6cd40540ddc765622283dea8a5e91c61fca433cd4e56846d2415ad4ce5f643
  Stored in directory: /home/fabbro/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pyspark
sc = pyspark.SparkContext(appName="mySparkApp")


23/01/16 14:56:24 WARN Utils: Your hostname, kerah resolves to a loopback address: 127.0.1.1; using 157.27.158.220 instead (on interface wlo1)
23/01/16 14:56:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/16 14:56:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/16 14:56:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


NameError: name 'SparkSession' is not defined

In [18]:
input_filename = '/home/fabbro/Documents/Big-Data/file.csv'
#input_filename = "/Users/dcarra/Desktop/OneDrive - Università degli Studi di Verona/courses/BigData/PhD_course/2021/lab_notebooks/datasets/short_stories_I.txt"
input_file = sc.textFile(input_filename)

#num_lines = input_file.count()
#print("The number of lines in the input file is:", num_lines)

print(input_file.take(1))
wordsList = input_file.filter(bool) \
            .flatMap(lambda line: line.split(" "))\
            .map(lambda w: w.lower())
wordsKeyValue = wordsList.flatMap(lambda w: (w,1))
print(wordsKeyValue.take(5))

['Title,Text']
['title,text', 1, 'accessiblecomputing,#redirect', 1, '[[computer']


In [ ]:
df = spark.read.csv('file.csv',header=True,escape=";")